# Experiment 000: Baseline

Establish baseline using pre-optimized submission from snapshots.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from decimal import Decimal
import json
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = float(str(center_x).replace('s', ''))
        self.center_y = float(str(center_y).replace('s', ''))
        self.angle = float(str(angle).replace('s', ''))
        
        initial_polygon = Polygon(zip(TX, TY))
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)
    
    def get_vertices(self):
        return list(self.polygon.exterior.coords)

print("ChristmasTree class defined successfully")

ChristmasTree class defined successfully


In [2]:
# Load pre-optimized submission
submission_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(submission_path)
print(f"Loaded submission with {len(df)} rows")
print(df.head(10))

Loaded submission with 20100 rows
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   
5  003_2      s0.641714640229075      s1.180458566613381   
6  004_0  s-0.324747789589372171   s0.132109978088185392   
7  004_1   s0.315354346242637695   s0.132109978063475492   
8  004_2   s0.324747789592379210  s-0.732109978069475531   
9  004_3  s-0.315354348134818330  s-0.732109978094185987   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  
5      s155.13405193710082  
6  s156.370622145636389178  
7  s156.370622269264089255  
8  s336.370622269264003990  
9  s336.3706221

In [3]:
# Parse submission and calculate score
def parse_value(val):
    """Parse value, removing 's' prefix if present"""
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def get_trees_for_n(df, n):
    """Get all trees for a given N value"""
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def get_bounding_box_side(trees):
    """Get the side length of the bounding box for a list of trees"""
    all_coords = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_coords.append(coords)
    all_coords = np.vstack(all_coords)
    min_x, min_y = all_coords.min(axis=0)
    max_x, max_y = all_coords.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def calculate_total_score(df):
    """Calculate total score for submission"""
    total = 0
    scores_by_n = {}
    for n in range(1, 201):
        trees = get_trees_for_n(df, n)
        if len(trees) != n:
            print(f"Warning: N={n} has {len(trees)} trees instead of {n}")
        side = get_bounding_box_side(trees)
        score = side**2 / n
        scores_by_n[n] = {'side': side, 'score': score}
        total += score
    return total, scores_by_n

print("Score calculation functions defined")

Score calculation functions defined


In [4]:
# Calculate score
print("Calculating total score...")
total_score, scores_by_n = calculate_total_score(df)
print(f"\nTotal Score: {total_score:.6f}")
print(f"\nTarget Score: 68.889699")
print(f"Gap to target: {total_score - 68.889699:.6f}")

# Show top contributors
print("\nTop 10 score contributors (highest s^2/n):")
sorted_scores = sorted(scores_by_n.items(), key=lambda x: x[1]['score'], reverse=True)
for n, data in sorted_scores[:10]:
    print(f"  N={n:3d}: side={data['side']:.4f}, score={data['score']:.4f}")

Calculating total score...



Total Score: 70.676102

Target Score: 68.889699
Gap to target: 1.786403

Top 10 score contributors (highest s^2/n):
  N=  1: side=0.8132, score=0.6612
  N=  2: side=0.9495, score=0.4508
  N=  3: side=1.1420, score=0.4347
  N=  5: side=1.4437, score=0.4168
  N=  4: side=1.2908, score=0.4165
  N=  7: side=1.6731, score=0.3999
  N=  6: side=1.5484, score=0.3996
  N=  9: side=1.8673, score=0.3874
  N=  8: side=1.7559, score=0.3854
  N= 15: side=2.3850, score=0.3792


In [5]:
# Copy submission to submission folder
import shutil

os.makedirs('/home/submission', exist_ok=True)
shutil.copy(submission_path, '/home/submission/submission.csv')
print(f"Copied submission to /home/submission/submission.csv")

# Verify the copy
df_check = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_check)} rows in submission")

Copied submission to /home/submission/submission.csv
Verified: 20100 rows in submission


In [6]:
# Save metrics
metrics = {
    'cv_score': total_score,
    'target_score': 68.889699,
    'gap': total_score - 68.889699
}

with open('/home/code/experiments/000_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Saved metrics to experiments/000_baseline/metrics.json")
print(f"\nFinal CV Score: {total_score:.6f}")

Saved metrics to experiments/000_baseline/metrics.json

Final CV Score: 70.676102
